In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as ssp

from DSI_Capstone_Steemit.utils.utils import(
    load_data_and_description,
)
from sklearn.model_selection import train_test_split

import os

data_directory = '../data/'
input_directory = os.path.join(data_directory,'networkx_votes')

def load_joblib(filename):
    return joblib.load(os.path.join(input_directory,filename))

data,feature_names,data_desc = load_data_and_description(data_type='tfidf')
data_desc['log total_payout_value'] = np.log(data_desc['total_payout_value'])



In [ ]:
data_desc

In [ ]:
idx = data_desc['depth'] != 0

idx.values

In [ ]:
data_desc = data_desc[idx]
data = data[idx.values]

In [ ]:
y.max()

In [ ]:
sns.distplot(data_desc['total_payout_value'])

In [ ]:
import joblib

In [ ]:
def add_network_features(df):

    hubs,authorities = load_joblib('hits')
    cluster = load_joblib('parts')
    pagerank = load_joblib('prank') 
    eig_cent = load_joblib('eig_cent') 
    core_k = load_joblib('core_k') 

    df['Cluster'] = df['author'].map(cluster)
    df.loc[:,'Cluster Condense'] = df['Cluster']
    df.loc[~df['Cluster'].isin([1,3,0,2,5,4]),'Cluster Condense'] = 'Other'
    df['Hubs'] = df['author'].map(hubs) * 10000
    df['Authorities'] = df['author'].map(authorities) * 10000
    df['Page Rank'] = df['author'].map(pagerank) * 10000
    df['Eigen Centrality'] = df['author'].map(eig_cent)* 10000
    df['Core K'] = df['author'].map(core_k)*10000
    return df
data_desc = add_network_features(data_desc)

network_cols = ['Page Rank','Cluster','Hubs','Authorities','Eigen Centrality']

In [ ]:

# Remove middle value articles

# idx1 = data_desc['log total_payout_value'] < 1.2
# idx2 = data_desc['log total_payout_value'] >2.5

# idx_not = (~idx1) & (~idx2)

data_desc = data_desc
data = data[:,:]
# y = data_desc['log total_payout_value'] >2.5

# For Regression
y = data_desc['total_payout_value']

value_counts = data_desc['category'].value_counts()
top_categories = value_counts.index[value_counts > np.percentile(data_desc['category'].value_counts(),97)]
idx = data_desc['category'].isin(top_categories)
data_desc['top category'] = idx.astype(int)

data_desc['top category listed'] = data_desc.ix[data_desc['top category'].values.astype(bool) ,'category']

data_desc['top category listed'] = data_desc['top category listed'].fillna('Other')


post_features = ['number of body tags',
                                   'number of body urls',
                                   'number of image urls',
                                   'number of body mentions',
                                   'number of image urls',
                                   'number of youtube urls',
                                   'language',
                                   'author_reputation_scaled',
                                   'number of steem counts',
                                'top category'] + network_cols


train_features = data_desc.ix[:,post_features].fillna(0)

train = pd.get_dummies(train_features)

# num_image_urls = train['number of image urls'].values[:,0]
# train.drop('number of image urls',axis = 1, inplace=True)

train['number of image urls'] = num_image_urls

training_names = train.columns

train_sparse = ssp.csr_matrix(train)
new_data = ssp.hstack([data,train_sparse])
train = new_data.tocsr()

# All samples
number_of_samples = train.shape[0]


# from sklearn.feature_selection import VarianceThreshold
# selector = VarianceThreshold()
# print data.shape
# train = selector.fit_transform(train)




X_train, X_test, y_train, y_test = train_test_split(
    train, y, test_size=0.33, random_state=42)


In [ ]:
y_train.max()

In [ ]:
idx = selector.get_support()


all_features = np.array(feature_names + list(training_names))

In [ ]:
print X_train.shape, len(all_features)

In [ ]:
for i in y_train:
    print i

In [ ]:

import xgboost as xgb

xgtrain = xgb.DMatrix(X_train, label=y_train)

def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha,learning_rate,reg_alpha,reg_lambda,scale_pos_weight):

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)
    params['learning_rate'] = max(learning_rate, 0)
    params['reg_alpha'] = max(reg_alpha, 0)
    params['reg_lambda'] = max(reg_lambda, 0)
    params['scale_pos_weight'] =  max(scale_pos_weight, 0)




    cv_result = xgb.cv(params, xgtrain, 
                       num_boost_round=num_boost_round, nfold=5,
             seed=random_state,
#              callbacks=[xgb.callback.early_stop(50)]
                      )

    return -cv_result['test-mae-mean'].mean()


num_boost_round = 10

random_state = 2016

# For Bayesian Optimization
num_iter = 30
init_points = 5
params = {
    'eta': 0.05,
    'silent': 0,
    'eval_metric': ['mae'],
    'objective':'reg:linear',
    'verbose_eval': True,
    'seed': random_state
}

In [ ]:
import time

start = time.time()
import xgboost as xgb
from bayes_opt import BayesianOptimization

xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (0.1, 1),
                                            'max_depth': (1, 15),
                                            'subsample': (0.5, 1),
                                            'gamma': (0, 10),
                                            'reg_lambda': (0, 10),
                                            'learning_rate': (0.001,1),
                                            'reg_alpha' : (0,2),
                                            'alpha':(0,2),
                                            'scale_pos_weight':(0,2)
                                            })
xgbBO.maximize(init_points=init_points, n_iter=num_iter)

print(xgbBO.res['max'])
print(xgbBO.res['all'])

end = time.time()
print 'Final Time'
print(end - start)



In [ ]:
# xgbBO.points_to_csv('bayesian_results.csv')
# import json
# with open('bayesian_results.json', 'w') as fp:
#     json.dump(xgbBO.res, fp)




In [ ]:
import matplotlib.pylab as plt
%matplotlib inline 
plt.plot(np.array(xgbBO.res['all']['values']))

In [ ]:
param_max = xgbBO.res['max']['max_params']

In [ ]:
param_max.pop('alpha')

In [ ]:
param_max['max_depth'] = int(param_max['max_depth'])

In [ ]:
model = xgb.XGBRegressor(**xgbBO.res['max']['max_params'])
model.fit(X_train, y_train)

In [ ]:
y_train

In [ ]:
predictions = model.predict(X_test)

In [ ]:
import seaborn as sns

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error

In [ ]:
predictions

In [ ]:
sns.regplot(predictions,y_test,
            scatter_kws={'alpha':0.5}
           )

print r2_score(y_pred=y_test,y_true=predictions)
print mean_absolute_error(y_pred=y_test,y_true=predictions)




In [ ]:
features = pd.DataFrame([model.feature_importances_,all_features]).T

In [ ]:
features.columns = ['Importance','Feature']

In [ ]:
features = features[features['Importance'] != 0]

In [ ]:
features = features.sort_values('Importance',
                                ascending=False).reset_index(drop = True)

In [ ]:
features